In [1]:
import pymc as pm
import numpy as np
import arviz as az

%load_ext lab_black
%load_ext watermark

# Gesell

Adapted from [Unit 9: Gesell.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit9/Gesell.odc).


Associated lecture video: Unit 9 lesson 6

## Problem statement

Gesell Developmental Schedules  measure  child development. The scoring is devised by the American child psychologist and pediatrician Arnold Gesell who founded the Clinic of Child Development at Yale in 1911 and directed it for many years.  The Gesell Developmental Schedules are a gauge of the status of a child's motor and language development and personal-social and adaptive behaviors.

For 21 children the age in months at which they first spoke and their Gesell Adaptive Score, which is the result of an aptitude test taken much later was recorded.  These data were originally collected by L. M. Linde of UCLA but were first published by M. R. Mickey, O. J. Dunn, and V. Clark, †Note on the use of stepwise regression in detecting outliers,† Computers and Biomedical Research, 1 (1967), pp. 105†111. 

The data have been used by several authors, e.g.,  N. R. Draper and J. A. John, †Influential observations and outliers in regression,† Technometrics, 23 (1981), pp. 21†26.

Let X be the age in in months a child speaks his/her first word and let Y be the Gesell adaptive score, a measure of a child†s aptitude (observed later on). 

Correlation of r = ¢0.640 indicates a  moderately strong negative relationship between age at first word spoken and Gesell score. How does the child†s aptitude change with how long it takes them to speak?

Are there any influential observations/potential outliers?

The solution follows a two-stage model:

a. Obtain mean ICPOs. Outside BUGS take reciprocal of mean ICPOs (these are CPOs) and consder them as an input for model B.

b. Read in and calibrate CPOs with posterior ordinates.

In [2]:
# fmt: off
x = np.array(
    (15, 26, 10, 9, 15, 20, 18, 11, 8, 20, 7, 9, 10, 11, 11, 10, 12, 42, 17, 11, 10)
)

y = np.array(
    (95, 71, 83, 91, 102, 87, 93, 100, 104, 94, 113, 96, 83, 84, 102, 100, 105, 57, 121, 86, 100)
)
# fmt: on

## Part A

In [3]:
with pm.Model() as model_a:
    # couldn't get .001 working, answers are pretty close with .003
    tau = pm.Gamma("tau", 1, 0.003)
    s2 = pm.Deterministic("s2", 1 / tau)
    b = pm.Normal("b", 0, tau=0.00001, shape=2)

    m = b[0] + b[1] * x
    r = y - m
    f = pm.Deterministic(
        "cpo", (tau / (2 * np.pi)) ** 0.5 * pm.math.exp(-0.5 * tau * r**2)
    )
    # what is the point of this when we want cpo anyways?
    icpo = pm.Deterministic("icpo", 1 / f)
    pm.Normal("likelihood", mu=m, tau=tau, observed=y)

    trace = pm.sample(10000, target_accept=0.99)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/Users/aaron/mambaforge/envs/pymc_env2/lib/python3.11/site-packages/multipledispatch/dispatcher.py:27: AmbiguityWarning: 
Ambiguities exist in dispatched function _unify

The following signatures may result in ambiguous behavior:
	[ConstrainedVar, object, Mapping], [object, ConstrainedVar, Mapping]
	[object, ConstrainedVar, Mapping], [ConstrainedVar, Var, Mapping]
	[object, ConstrainedVar, Mapping], [ConstrainedVar, object, Mapping]
	[object, ConstrainedVar, Mapping], [ConstrainedVar, Var, Mapping]


Consider making the following additions:

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)
  warn(warning_text(dispatcher.name, ambiguities), AmbiguityWarning)
Multiprocess sampling (4 chains in 4

Sampling 4 chains for 1_000 tune and 10_000 draw iterations (4_000 + 40_000 draws total) took 20 seconds.


In [4]:
icpo_df = az.summary(trace, var_names="icpo", round_to=5, filter_vars="like")

ICPO results match BUGS.

In [5]:
icpo_df

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
icpo[0],28.49252,4.79883,20.22630,37.39362,0.04013,0.02904,15402.75882,16354.92217,1.00005
icpo[1],48.15250,25.65207,22.43135,87.06060,0.17364,0.12278,22258.37282,25279.19437,1.00003
icpo[2],92.68060,50.43700,37.18320,174.94619,0.34648,0.24500,19756.34715,23498.46900,1.00028
icpo[3],40.55863,10.54957,24.90041,59.54200,0.08012,0.05665,18132.83162,23275.09662,1.00018
icpo[4],40.82201,8.62304,27.08866,56.88327,0.04957,0.03706,34540.14826,26037.10808,1.00007
icpo[5],28.35416,4.95485,20.11566,37.65425,0.04289,0.03101,14453.53955,15544.56368,1.00009
icpo[6],29.66543,5.18721,20.99113,39.42179,0.04159,0.03006,16901.49990,17924.92998,0.99999
icpo[7],28.93442,4.98993,20.55201,38.30049,0.04186,0.03030,15391.33197,16437.96768,1.00004
icpo[8],29.82527,5.61253,20.57707,40.09568,0.04789,0.03465,15100.17290,16558.27920,1.00009
icpo[9],34.95297,7.73802,22.80536,49.01526,0.05294,0.03866,24268.30824,23598.83146,1.00001


In [6]:
cpo_means = 1 / icpo_df["mean"].to_numpy()
cpo_means

array([0.03509693, 0.02076735, 0.01078974, 0.02465567, 0.02449659,
       0.03526819, 0.03370927, 0.03456091, 0.03352862, 0.02860987,
       0.01922633, 0.03313354, 0.01078974, 0.01494588, 0.03237407,
       0.03515556, 0.02518198, 0.0150638 , 0.00016897, 0.01912319,
       0.03515556])

## Part B

In [7]:
mlcpo = -2 * np.log(cpo_means)
MPQ = np.sum(mlcpo)

In [9]:
with pm.Model() as model_b:
    tau = pm.Gamma("tau", 1, 0.002)
    s2 = pm.Deterministic("s2", 1 / tau)
    b = pm.Normal("b", 0, tau=0.00001, shape=2)

    m = b[0] + b[1] * x
    r = y - m
    f = pm.Deterministic(
        "f", (tau / (2 * np.pi)) ** 0.5 * pm.math.exp(-0.5 * tau * r**2)
    )
    c = pm.Deterministic("c", cpo_means / f)

    g1 = pm.Deterministic("g1", c * pm.math.log(c))
    g2 = -pm.math.log(c)
    g3 = 0.5 * np.absolute((c - 1))
    g4 = (c - 1) ** 2

    pm.Deterministic("s_g1", pm.math.sum(g1))
    pm.Deterministic("s_g2", pm.math.sum(g2))
    pm.Deterministic("s_g3", pm.math.sum(g3))
    pm.Deterministic("s_g4", pm.math.sum(g4))

    pm.Normal("likelihood", mu=m, tau=tau, observed=y)

    traceb = pm.sample(2000, target_accept=0.95, idata_kwargs=dict(log_likelihood=True))

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, b]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 2 seconds.


In [10]:
az.summary(traceb, var_names="c", filter_vars="like", kind="stats")

,mean,sd,hdi_3%,hdi_97%
c[0],1.000,0.166,0.711,1.312
c[1],0.999,0.565,0.465,1.804
c[2],0.998,0.527,0.374,1.844
c[3],1.000,0.261,0.633,1.485
c[4],1.002,0.214,0.671,1.409
c[5],1.000,0.172,0.699,1.318
c[6],1.000,0.174,0.702,1.318
c[7],1.000,0.170,0.706,1.314
c[8],1.000,0.187,0.684,1.338
c[9],1.001,0.222,0.648,1.401


In [11]:
az.summary(traceb, var_names="s_g", filter_vars="like", kind="stats")

,mean,sd,hdi_3%,hdi_97%
s_g1,4.634,107.366,-2.695,9.572
s_g2,2.285,1.227,0.080,3.804
s_g3,2.949,8.248,1.138,5.396
s_g4,260.023,15907.269,0.561,17.380


In [12]:
az.summary(traceb, var_names="g1", kind="stats")

,mean,sd,hdi_3%,hdi_97%
g1[0],0.014,0.175,-0.253,0.339
g1[1],0.104,1.061,-0.362,1.029
g1[2],0.107,0.784,-0.368,1.122
g1[3],0.031,0.309,-0.317,0.545
g1[4],0.023,0.238,-0.288,0.446
g1[5],0.014,0.184,-0.262,0.344
g1[6],0.015,0.186,-0.256,0.352
g1[7],0.014,0.180,-0.260,0.338
g1[8],0.017,0.203,-0.274,0.370
g1[9],0.024,0.254,-0.303,0.442


In [13]:
az.waic(traceb)

/Users/aaron/mambaforge/envs/pymc_env2/lib/python3.11/site-packages/arviz/stats/stats.py:1645: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(


Computed from 8000 posterior samples and 21 observations log-likelihood matrix.

          Estimate       SE
elpd_waic   -82.67     5.03
p_waic        3.38        -

There has been a warning during the calculation. Please check the results.

In [14]:
%watermark -n -u -v -iv -p aesara,aeppl

Last updated: Fri Feb 03 2023

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.9.0

aesara: 2.8.10
aeppl : 0.1.1

pymc : 5.0.1
arviz: 0.14.0
numpy: 1.24.1

